In [2]:
from query_vroom import *
from query_osrm import OsrmQuery
import pandas as pd
import numpy as np
import json
from clustering import DBScanClusterer

with open(r"C:\Users\clement.lork\Documents\Sembwaste_Routing\swm_route_optimization\azure_repos\admin_frontend\data1.json") as f:
  data = json.loads(f.read())

In [3]:
from solve_no_clusters import solve_nocluster
output = solve_nocluster(data)

print(output)

/table/v1/car/103.803251749,1.2975874959;103.8015381601,1.2991045323;103.8012557253,1.2981694472;103.8029222855,1.2970478898;103.8034980162,1.2949669;103.8059849537,1.2938049;103.8042774846,1.2956737;103.80266735,1.2918678;103.80247,1.29026;103.80227,1.29792;103.8006147809,1.2962227643;103.8009496073,1.2913810053;103.806630449,1.290363;103.8077590945,1.2878833;103.8151952073,1.2893498964;103.81321,1.28738;103.8163366739,1.2862738254;103.8152417375,1.28587615;103.81581,1.28685;103.81639,1.28721;103.8169,1.28664;103.81701,1.2879;103.8155536,1.28833;103.81552,1.28803;103.81609,1.2883;103.81649,1.28885;103.81703,1.28894;103.81758,1.28855;103.81937,1.28794;103.81935,1.28729;103.81912,1.28669;103.81888,1.28727;103.81884,1.28592;103.81865,1.28594;103.8107121615,1.2931516;103.8121070076,1.29298105;103.8114222239,1.29438085;103.8109724232,1.2944609;103.8113931236,1.2931142;103.8116368241,1.2938599;103.8121025838,1.29372165;103.8121812035,1.2942066;103.81132,1.29287;103.81039,1.29442;103.81033,1

### Add in VehicleTypes data

In [4]:
data["vehicleTypes"] = [{'id' : 0,'Name' : "4x2",
     'Capacity' : 8000,
     'Range' : 300000},
     {'id' : 1,
     'Name' : "6x4",
     'Capacity' : 12000,
     'Range' : 300000},
     {'id' : 2,
     'Name' : "8x2",
     'Capacity' : 16000,
     'Range' : 300000}
]

### Convert to DataFrame

In [5]:
solverConfig = data["solverConfig"]
points = data["pointsToAssign"]
fleet = data["vehiclesInvolved"]
routes = data["routesInvolved"]
vtypes = data['vehicleTypes']

solverConfig["useClusters"] = False

In [6]:
points = pd.DataFrame.from_dict(points)
fleet = pd.DataFrame.from_dict(fleet)
routes = pd.DataFrame.from_dict(routes)
vtypes = pd.DataFrame.from_dict(vtypes)

In [7]:
endpoint = "http://localhost:5001"
querycar = OsrmQuery(endpoint, 'car')
route_response = querycar.find_route(1.2984382,103.8047524,1.2980271,103.8050503,u_turn_allowed=True)
route_response["routes"][0]["duration"]/60

0.26666666666666666

In [8]:
points.columns

Index(['id', 'House', 'Street', 'Alias', 'Postal_Code', 'Premises',
       'Direct Collection', 'Allowed_Veh', 'Ideal_Vehicle_Type',
       'Service_Time_min', 'Vehicle_Type_Allowed', 'Min_Collection_Time_hrs',
       'Max_Collection_Time_hrs', 'Time_Collected_hrs', 'Tonnage_kg',
       'Manpower', 'Remarks', 'Route_ID', 'Vehicle_ID', 'Lat_User',
       'Long_User', 'Fatigue', 'Leg', 'Node_Osmid', 'Node_Latlong',
       'User_LongLat', 'Poly_Cluster', 'Network_Cluster',
       'Network_Vehicle_Type_Allowed', 'User_Timing', 'Cluster_Start',
       'Cluster_End', 'Completed'],
      dtype='object')

In [9]:
vtypes2str = dict(zip(vtypes.id.tolist(),vtypes.Name.tolist()))
vtypes2id = dict(zip(vtypes.Name.tolist(),vtypes.id.tolist()))

In [10]:
# convert Vehicle_Type back to string
fleet["Vehicle_Type"] = fleet["Vehicle_Type"].apply(lambda x: vtypes2str[x])

In [11]:
# check if a vehicle is servicing point.route_id, if true set allowed vehicle as the vehicle id

rid2vid = dict(zip(fleet.Route_ID.tolist(),fleet.id.tolist()))
vt2vid = dict(zip(fleet.Vehicle_Type.tolist(),fleet.id.tolist()))

vt2vid  = {}

for idx, vtype in enumerate(fleet['Vehicle_Type']):
    if vtype not in vt2vid :
        vt2vid[vtype] = []
        vt2vid[vtype].append(fleet["id"][idx])
    else:
        vt2vid[vtype].append(fleet["id"][idx])
print(vt2vid)


{'8x2': [0], '6x4': [2]}


### Find vehicle skils before clustering

In [12]:
vehicle_skills = {}
for vid in fleet.id:
    vehicle_skills[vid] = []

for idx, p in points.iterrows():
    # if route_id of points is inside vehicle, append p.id to vehicle
    rid = p["Route_ID"]
    allowed_veh = []
    if rid in rid2vid.keys():
        vid = rid2vid[rid]
        vehicle_skills[vid].append(p['id'])
        points.loc[idx,"Allowed_Veh"] = str(vid)
    else:
        # if vehicles does not have assigned routes, fall back to checking vehicle_type_allowed tags
        allowed_veh_list = []
        for vt in p["Vehicle_Type_Allowed"].split(","):
            if vt in vt2vid.keys():
                allowed_veh_list += [str(i) for i in vt2vid[vt]]
                for vid in vt2vid[vt]:
                    vehicle_skills[vid].append(p['id'])
        allowed_veh = ",".join(allowed_veh_list)
        points.loc[idx,"Allowed_Veh"] = allowed_veh

### Cluster Points again if using clustering

In [13]:
cluster_midpoints = list(zip(points["Lat_User"].tolist(), points["Long_User"].tolist()))

### Add Start Point and End Points

In [14]:
tuas_osmid, tuas_latlong = querycar.find_nearest(1.33469, 103.64579)
ip_osmid, ip_latlong = querycar.find_nearest(1.29607, 103.62173)
work_timewindows = [7*60*60,24*60*60]

cluster_latlongs = cluster_midpoints.copy()
cluster_latlongs.append(tuas_latlong)
print("tuas@",len(cluster_latlongs)-1)
depot_idx = len(cluster_latlongs)-1
cluster_latlongs.append(ip_latlong)
print("ip@",len(cluster_latlongs)-1)
ip_idx = len(cluster_latlongs)-1

tuas@ 126
ip@ 127


### Calculate distance matrix

In [15]:
dist_matrix, time_matrix = querycar.find_matrix(cluster_latlongs)
time_matrix = time_matrix*60
# time_matrix[0:500,500] = time_matrix[0:500,500]+10*60*60
# time_matrix[500,0:500] = time_matrix[500,0:500]+10*60*60

dist_matrix = dist_matrix.astype("int64").tolist()
time_matrix = time_matrix.astype("int64").tolist()

/table/v1/car/103.803251749,1.2975874959;103.8015381601,1.2991045323;103.8012557253,1.2981694472;103.8029222855,1.2970478898;103.8034980162,1.2949669;103.8059849537,1.2938049;103.8042774846,1.2956737;103.80266735,1.2918678;103.80247,1.29026;103.80227,1.29792;103.8006147809,1.2962227643;103.8009496073,1.2913810053;103.806630449,1.290363;103.8077590945,1.2878833;103.8151952073,1.2893498964;103.81321,1.28738;103.8163366739,1.2862738254;103.8152417375,1.28587615;103.81581,1.28685;103.81639,1.28721;103.8169,1.28664;103.81701,1.2879;103.8155536,1.28833;103.81552,1.28803;103.81609,1.2883;103.81649,1.28885;103.81703,1.28894;103.81758,1.28855;103.81937,1.28794;103.81935,1.28729;103.81912,1.28669;103.81888,1.28727;103.81884,1.28592;103.81865,1.28594;103.8107121615,1.2931516;103.8121070076,1.29298105;103.8114222239,1.29438085;103.8109724232,1.2944609;103.8113931236,1.2931142;103.8116368241,1.2938599;103.8121025838,1.29372165;103.8121812035,1.2942066;103.81132,1.29287;103.81039,1.29442;103.81033,1

### Add Vehicles

In [16]:
fleet

,id,Name,Vehicle_Type,Route_ID,Max_Trips,Manpower,Depot,Break_Time,Current_Lat,Current_Long,Current_Tonnage,Start_Time,End_Time,vehicleCapacity,vehicleRange
0,0,XE6239D,8x2,0,3,2,0,12,1.3344,103.64579,0,00:00,00:00,16000,120
1,2,XE5596D,6x4,2,3,2,0,12,1.3344,103.64579,0,00:00,00:00,12000,120


In [17]:
vehicles = []
for idx, veh in fleet.iterrows():
    v = Vehicle(veh['id'], 'car', 
                start =None, start_index=int(depot_idx), 
                end =None, end_index=int(depot_idx), 
                capacity=[int(veh["vehicleCapacity"])], costs=None,
                skills = vehicle_skills[veh['id']],
                time_window=[7*60*60,18*60*60], max_tasks=None,
                max_travel_time=None,
                max_distance=300000).to_dict()
    vehicles.append(v)

In [18]:
data.keys()

dict_keys(['pointsToAssign', 'vehiclesInvolved', 'routesInvolved', 'depotData', 'solverConfig', 'vehicleTypes'])

In [19]:
cluster_load = points["Tonnage_kg"].tolist()
cluster_service_time = points["Service_Time_min"].tolist()
cluster_tw = list(zip(points["Min_Collection_Time_hrs"].tolist(), points["Max_Collection_Time_hrs"]))

### Add pickup delivery

In [20]:
shipments = []
for idx in range(0,len(cluster_midpoints)):
    amount = [cluster_load[idx]]
    skills = [points.iloc[idx]['id']]
    p = points.iloc[idx]
    pickup_step = Shipment_Step(id=idx,
                                description = (str(p["House"])+" "+str(p["Street"])+" "+str(p["Alias"])).replace("None","").rstrip(" ").lstrip(" "),
                                location = None,
                                location_index = idx,
                                setup = None,
                                service = int(cluster_service_time[idx])*60,
                                time_windows = [[int(cluster_tw[idx][0])*60*60, int(cluster_tw[idx][1])*60*60]]).to_dict()
    delivery_step = Shipment_Step(id=len(cluster_midpoints)+2+idx,
                                  description = "IP",
                                  location= None,
                                  location_index= ip_idx,
                                  setup = 20*60,
                                  service= None,
                                  time_windows = [[7*60*60, 24*60*60]]).to_dict()
    ship = Shipment(pickup_step=pickup_step, delivery_step=delivery_step, amount=amount, skills=skills, priority=0).to_dict()
    shipments.append(ship)


    matrix = Matrix(duration_matrix = time_matrix, distance_matrix=None, cost_matrix=None).to_dict()


In [21]:
data = sendData(jobs=[],shipments=shipments,vehicles=vehicles,matrices=matrix).to_dict()

In [22]:
data = json.dumps(data, indent=2, default=int)

In [23]:
data = json.loads(data)

In [24]:
output = solve(data)

In [25]:
output.keys()

dict_keys(['code', 'summary', 'unassigned', 'routes'])

In [26]:
for u in output["unassigned"]:
    if u['type'] == 'pickup':
        print(u)

{'id': 80, 'location_index': 80, 'type': 'pickup', 'description': '1 JALAN KUKOH'}
{'id': 79, 'location_index': 79, 'type': 'pickup', 'description': '52 CHIN SWEE RD Seacare Hotel'}
{'id': 78, 'location_index': 78, 'type': 'pickup', 'description': '333 KRETA AYER RD'}


In [27]:
output["summary"]

{'cost': 18900,
 'routes': 2,
 'unassigned': 6,
 'delivery': [51760],
 'amount': [51760],
 'pickup': [51760],
 'setup': 4800,
 'service': 43380,
 'duration': 18900,
 'waiting_time': 0,
 'priority': 0,
 'violations': [],
 'computing_times': {'loading': 2, 'solving': 1590, 'routing': 0}}

In [28]:
for k in range(0,len(output['routes'])):
    route1 = []
    for i in output['routes'][k]["steps"]:
        if i["location_index"] == depot_idx:
            route1.append("DP")
        elif i["location_index"] > depot_idx:
            route1.append("IP")

        else:
            route1.append(i["location_index"])


    route1 = [route1[i] for i in range(len(route1)) if i == 0 or route1[i] != route1[i - 1]]
    print(route1)

['DP', 7, 8, 4, 2, 10, 11, 12, 42, 38, 34, 35, 40, 41, 39, 37, 46, 48, 47, 45, 43, 44, 36, 50, 'IP', 13, 17, 16, 51, 49, 28, 31, 29, 30, 33, 32, 20, 18, 19, 22, 23, 24, 25, 26, 27, 21, 15, 14, 5, 6, 3, 9, 0, 1, 'IP', 'DP']
['DP', 97, 96, 121, 119, 118, 115, 114, 113, 116, 109, 117, 110, 112, 111, 125, 124, 123, 122, 61, 60, 56, 53, 52, 54, 59, 63, 57, 81, 107, 106, 104, 103, 105, 100, 102, 101, 120, 108, 99, 95, 92, 94, 93, 91, 98, 'IP', 65, 64, 62, 58, 55, 82, 86, 90, 85, 68, 66, 83, 67, 73, 72, 71, 70, 69, 77, 76, 75, 74, 84, 87, 88, 89, 'IP', 'DP']


In [29]:
output["unassigned"][1]

{'id': 80,
 'location_index': 80,
 'type': 'pickup',
 'description': '1 JALAN KUKOH'}

In [30]:
output["routes"][0]["steps"][1]

{'type': 'pickup',
 'description': '163  STIRLING ROAD',
 'location_index': 7,
 'id': 7,
 'setup': 0,
 'service': 1200,
 'waiting_time': 0,
 'job': 7,
 'load': [1500],
 'arrival': 26520,
 'duration': 1320,
 'violations': []}

In [31]:
output["routes"][0]["steps"][0]

{'type': 'start',
 'location_index': 126,
 'setup': 0,
 'service': 0,
 'waiting_time': 0,
 'load': [0],
 'arrival': 25200,
 'duration': 0,
 'violations': []}

In [32]:
33//6

5

In [33]:

for i in output["routes"][0]["steps"]:
    if i['type'] == "pickup":
        h = i["arrival"]//(60*60)
        mm = round(i["arrival"]%(60*60)/(60*60)*60)

        s="<div class='list-group-item'>{k}</div>".format(k= str(h)+":"+str(mm)+"<br/>" + i['description']+"<br/>"+str(i['load'][0])+"kg")
        print(s)

<div class='list-group-item'>7:22<br/>163  STIRLING ROAD<br/>1500kg</div>
<div class='list-group-item'>7:43<br/>166 STIRLING ROAD<br/>3500kg</div>
<div class='list-group-item'>8:6<br/>143 - 150 MEI LING STREET (special equipment for CRC)<br/>4250kg</div>
<div class='list-group-item'>8:28<br/>580 - 590 STIRLING ROAD<br/>5000kg</div>
<div class='list-group-item'>8:36<br/>22A QUEENSWAY Special Ops<br/>6500kg</div>
<div class='list-group-item'>8:41<br/>260 - 264 QUEENSWAY<br/>6740kg</div>
<div class='list-group-item'>8:46<br/>358 - 360 ALEXANDRA ROAD Shell & Caltex<br/>6980kg</div>
<div class='list-group-item'>8:51<br/>57 DAWSON RD Dawson Place<br/>7980kg</div>
<div class='list-group-item'>9:11<br/>59 STRATHMORE AVE<br/>8280kg</div>
<div class='list-group-item'>9:15<br/>58 STRATHMORE AVE<br/>8580kg</div>
<div class='list-group-item'>9:19<br/>60 STRATHMORE AVE<br/>8880kg</div>
<div class='list-group-item'>9:24<br/>61B STRATHMORE AVE<br/>9180kg</div>
<div class='list-group-item'>9:28<br/>61C